# Householder QR

The numerical downfall of the Gram-Schmidt algorithm lies in performing multiple operations and hoping to end up with a unitary resulting $Q$. The key to a stable algorithm is to go the other way: operate orthogonally (unitarily) on $A$ in order to produce a triangular $R$. It's a lot like Gaussian elimination, proceeding one column at a time. Instead of elementary row operations, though, we want orthogonal transformations. Two natural sources of these are reflections and rotations. For general matrices, the reflection approach is more efficient. 

The key step is to find, given $x$, a unitary $F$ such that $Fx=\alpha e_1$ for a scalar $\alpha$. Since $F$ preserves the 2-norm, $\alpha= \pm \|x\|_2$. One can simply exhibit the solution. Define $v=\alpha e_1 - x$ and then set $$ F = I - 2 \frac{vv^*}{v^*v}.$$

In [1]:
x = randn(5,1);  alpha = norm(x);
I = eye(5);
v = alpha*[1;zeros(4,1)] - x;
F = I - 2*(v*v')/(v'*v)


F =

    0.1735    0.5919   -0.7291    0.2783    0.1029
    0.5919    0.5761    0.5221   -0.1993   -0.0737
   -0.7291    0.5221    0.3569    0.2455    0.0908
    0.2783   -0.1993    0.2455    0.9063   -0.0346
    0.1029   -0.0737    0.0908   -0.0346    0.9872



In [2]:
F'*F-I


ans =

   1.0e-15 *

   -0.3331    0.1011   -0.1509    0.0298    0.0162
    0.1011   -0.1110    0.1037   -0.1208   -0.0255
   -0.1509    0.1037   -0.1110    0.0807    0.0253
    0.0298   -0.1208    0.0807         0   -0.0129
    0.0162   -0.0255    0.0253   -0.0129         0



In [3]:
format long, F*x


ans =

   3.098316228803490
   0.000000000000000
  -0.000000000000000
   0.000000000000000
   0.000000000000000



So we proceed through columns $j=1,2,\ldots$ of $A$. In column $j$, we want to introduce zeros below the diagonal, so $x$ is drawn from rows $j$ to $m$. For example, with $j=1$ (note that $v$ is normalized to simplify the formula for the reflector):

In [4]:
A = randi(5,6,3);  
v = -A(:,1);  % -x
v(1) = v(1) + norm(v);
v = v/norm(v);
F = eye(6) - 2*(v*v');
A = F*A


A =

   8.062257748298549   7.069979871584883   7.194014606174091
  -0.000000000000000   4.413790902184621   3.378664816728708
                   0   2.120686353276931   4.067997225093062
  -0.000000000000001   3.534477255461551   2.446662041821769
  -0.000000000000001  -0.465522744538449  -0.553337958178231
  -0.000000000000000   2.706895451092310   4.689332408364353



Now we can proceed with $j=2$. 

In [5]:
v = -A(2:end,2);  % -x
v(1) = v(1) + norm(v);
v = v/norm(v);
F = eye(5) - 2*(v*v');
A(2:end,2:end) = F*A(2:end,2:end);
A


A =

   8.062257748298549   7.069979871584883   7.194014606174091
  -0.000000000000000   6.634409138377328   6.803689763019602
                   0   0.000000000000000   0.797104666953309
  -0.000000000000001                   0  -3.004825555077819
  -0.000000000000001  -0.000000000000000   0.164672453122212
  -0.000000000000000                   0   0.514285884884421



And so on. Once $j$ sweeps from 1 to $n$, the matrix will be transformed into the $R$ we seek. If we properly accumulate the actions of the reflectors, we end up with $Q$ as well (the full one or the thin one, as we choose).  

I have left out a detail that is important to the stability. A full code is part of the exercises. 

## Q-less QR

When we solved the linear system $Ax=b$ with the factorization, we transformed it to the equivalent system $Rx=Q^*b$. Thus (as is often the case) we don't really want $Q$, or even $Q^*$, but the ability to compute $Q^*b$ for given $b$. The Householder method computes $F_n\cdots F_1A=R,$ where each $F_j$ contains a reflector, so $$Q^*b=F_n^*\cdots F_1^*b=F_n\cdots F_1b.$$ That is, we just have to apply the reflectors to $b$ (in rows $j$ to $m$ for $F_j$). This is more efficient than constructing the $Q$ itself (which can be obtained by letting $b$ take on the columns of an identity matrix). You can get this more efficient form by calling `qr` differently. 